<a href="https://colab.research.google.com/github/kangwonlee/pytorch-ibm-coursera/blob/main/week06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Hello PyTorch 👋🏻



references
* https://www.coursera.org/learn/deep-neural-networks-with-pytorch/
* https://github.com/damounayman/Deep-Neural-Networks-with-PyTorch/blob/main/Week1/1D_tensors.ipynb



## week 6



### 9.1 Convolution



### 9.2 Activation Functions and Max Polling



### 9.3 Multiple Input and Output Channels



### 9.4 Convolutional Neural Network



### 9.5 Torch-Vision Models

